In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import time
import pickle

In [18]:
root = "../../../../../"

In [19]:
df = pd.read_csv(root + "datasets/multiclass/processed/CICDDoS_basic.csv", index_col=[0])

In [20]:
df.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,1.0,2.0,0.0,2736.0,0.0,1368.0,1368.0,1368.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
1,1.0,2.0,0.0,2650.0,0.0,1325.0,1325.0,1325.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
2,2.0,2.0,0.0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
3,46.0,2.0,0.0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS
4,1.0,2.0,0.0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DrDoS_DNS


In [21]:
X = df.drop(columns=[' Label'])
y = df[' Label']

In [22]:
splits = 10
fs_times = []
train_times = []
fit_times = []
number_features = []
predict_times = []
test_sizes = []
score_times = []
test_accuracies = []
test_precisions = []
test_recalls = []
test_f1_scores = []

skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X, y):
    X_train,  X_test = X.iloc[train_index], X.iloc[test_index]
    y_train,  y_test = y.iloc[train_index], y.iloc[test_index]

    start = time.time()
    # Feature Selection
    fs_start = time.time()
    corrmat = X_train.corr()
    corrmat = corrmat.abs().unstack() # absolute value of corr coef
    corrmat = corrmat.sort_values(ascending=False)
    corrmat = corrmat[corrmat > 0.9]
    corrmat = corrmat[corrmat < 1]
    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']

    grouped_feature_ls = []
    correlated_groups = []

    for feature in corrmat.feature1.unique():    
        if feature not in grouped_feature_ls:
            # find all features correlated to a single feature
            correlated_block = corrmat[corrmat.feature1 == feature]
            grouped_feature_ls = grouped_feature_ls + list(correlated_block.feature2.unique()) + [feature]
            # append the block of features to the list
            correlated_groups.append(correlated_block)

    group_head = []

    for group in correlated_groups:
        group_head.append(group.iloc[0,0])

    features_to_remove = [feature for feature in grouped_feature_ls if feature not in group_head]

    X_train.drop(columns=features_to_remove, inplace=True)
    X_test.drop(columns=features_to_remove, inplace=True)
    fs_end = time.time()
    # Training the model
    train_start = time.time()
    clf_xgb = xgb.XGBClassifier(seed=42)
    # clf_xgb.fit(X_train, 
    #             y_train,
    #             # verbose=True,
    #             ## the next three arguments set up early stopping.
    #             early_stopping_rounds=5,
    #             eval_metric='logloss',
    #             eval_set=[(X_test, y_test)])
    clf_xgb.fit(X_train, y_train)
    train_end = time.time()
    end = time.time()

    fs_times.append(fs_end - fs_start)
    train_times.append(train_end - train_start)
    fit_times.append(end - start)

    number_features.append(len(X_train.columns))

    start = time.time()
    y_pred = clf_xgb.predict(X_test)
    end = time.time()
    predict_times.append(end - start)

    test_sizes.append(len(y_pred))

    start = time.time()
    test_accuracies.append(accuracy_score(y_test, y_pred))
    test_precisions.append(precision_score(y_test, y_pred, average="macro"))
    test_recalls.append(recall_score(y_test, y_pred, average='macro'))
    test_f1_scores.append(f1_score(y_test, y_pred, average='macro'))
    end = time.time()
    score_times.append(end - start)

fs_times = np.array(fs_times)
train_times = np.array(train_times)
fit_times = np.array(fit_times)
number_features = np.array(number_features)
predict_times = np.array(predict_times)
test_sizes = np.array(test_sizes)
test_accuracies = np.array(test_accuracies)
test_precisions = np.array(test_precisions)
test_recalls = np.array(test_recalls)
test_f1_scores = np.array(test_f1_scores)
score_times = np.array(score_times)

c:\Users\pedro\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:54:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\pedro\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:54:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\pedro\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[02:54:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\pedro\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:54:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\pedro\miniconda3\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\pedro\miniconda3\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:54:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [23]:
pfm = pd.DataFrame([test_accuracies, test_precisions, test_recalls, test_f1_scores,
                    fit_times, fs_times, train_times, predict_times, score_times, 
                    number_features, test_sizes])
pfm = pfm.T
pfm.columns = ["Accuracy", "Precision", "Recall", "F1_Score", 
                "Fit_Time", "FS_Time", "Train_Time","Predict_Time", "Score_Time", 
                "Number_Features", "Test_Size"]

In [24]:
filename = root + "pickles/multiclass_categorical/cross_validation/label_independent/corr.pkl"
outfile = open(filename, 'wb')
pickle.dump(pfm, outfile)
outfile.close()

In [25]:
pfm

,Accuracy,Precision,Recall,F1_Score,Fit_Time,FS_Time,Train_Time,Predict_Time,Score_Time,Number_Features,Test_Size
0,0.744318,0.750008,0.744318,0.729242,3.248998,0.081002,3.167996,0.014001,0.027000,37.0,528.0
1,0.763258,0.778292,0.763258,0.752428,3.202003,0.137007,3.064997,0.009997,0.019996,35.0,528.0
2,0.746212,0.744440,0.746212,0.738833,3.563123,0.113998,3.449124,0.011990,0.028006,38.0,528.0
3,0.717803,0.714703,0.717803,0.709820,3.432001,0.138008,3.293993,0.012999,0.027005,37.0,528.0
4,0.729167,0.732460,0.729167,0.720097,3.744003,0.133008,3.610995,0.011997,0.028000,38.0,528.0
5,0.753788,0.760096,0.753788,0.742941,3.635010,0.135007,3.500002,0.011994,0.027997,37.0,528.0
6,0.753788,0.762215,0.753788,0.737890,3.400003,0.138007,3.261995,0.011997,0.027997,35.0,528.0
7,0.740530,0.738361,0.740530,0.734981,3.692002,0.135005,3.556997,0.010999,0.026998,37.0,528.0
8,0.729167,0.730206,0.729167,0.717306,3.685007,0.133004,3.552003,0.012991,0.029001,37.0,528.0
9,0.702652,0.706280,0.702652,0.696373,3.586007,0.134007,3.452000,0.013992,0.028003,37.0,528.0


In [26]:
pfm["Number_Features"].mean()

36.8

In [27]:
pfm["Number_Features"].mean().round()

37.0

In [28]:
pfm["Number_Features"].mean().astype("int")

36

In [29]:
pfm["Fit_Time"].mean()

3.518815588951111